In [72]:
from load_results import load_result_dataset
import pandas as pd

pn1 = 'full_fine_tuning_50epochs_edge_paper_final2'
pn2 = 'full_fine_tuning_50epochs_paper_final2'
final_data1 = load_result_dataset(pn1, pn2)
final_data1 = [{**d, 'ft_strategy': 'FFT (50 epochs)'} for d in final_data1]
df1 = pd.DataFrame(final_data1)

pn1 = 'full_fine_tuning_5epochs_edge_article1'
pn2 = 'full_fine_tuning_5epochs_article1'
final_data2 = load_result_dataset(pn1, pn2)
final_data2 = [{**d, 'ft_strategy': 'FFT (5 epochs)'} for d in final_data2]
df2 = pd.DataFrame(final_data2)

pn1 = 'linearprobe_50epochs_edge_paper_final2'
pn2 = 'linearprobe_50epochs_paper_final2'
final_data3 = load_result_dataset(pn1, pn2)
final_data3 = [{**d, 'ft_strategy': 'LP (50 epochs)'} for d in final_data3]
df3 = pd.DataFrame(final_data3)

df = pd.concat([ df1, df2, df3 ], axis=0, ignore_index=False) 

final_data = []
final_data.extend(final_data1)
final_data.extend(final_data2)
final_data.extend(final_data3)

../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_

In [73]:
metrics = [ 'clean_acc', 'Linf_acc', 'L2_acc', 'L1_acc', 'common_acc' ]

datas = [ 'stanford_cars',
        'oxford-iiit-pet',
        'caltech101' ,
        'flowers-102' ,
        'fgvc-aircraft-2013b',
        'uc-merced-land-use-dataset'
         ]

resultats = []

from process_database import global_grouped_dataset
grouped_df = global_grouped_dataset(df)

for dataset in datas:

    dataset_result = []
    for metric in metrics:
        dataset_result.append( grouped_df[ (dataset,metric) ].rank(ascending=False, method='min') ) 

    dataset_result = pd.DataFrame(dataset_result).T.reset_index()
    dataset_result['total_score_{}'.format(dataset)] = dataset_result[(dataset, )].sum(axis=1)
    del dataset_result[(dataset,)]
    resultats.append(dataset_result)

resultats = pd.concat(resultats)
resultats.to_csv("./discontinuity.csv")
# dataet_result[('stanford_cars', )] #.drop( metrics )
# dataset_result[ (dataset,metrics) ].sum(axis=1)

/var/folders/v7/3s0lms795672_f7_mh2x6bcr0000gn/T/ipykernel_88335/2001734794.py:23: PerformanceWarning: indexing past lexsort depth may impact performance.
  dataset_result['total_score_{}'.format(dataset)] = dataset_result[(dataset, )].sum(axis=1)
/var/folders/v7/3s0lms795672_f7_mh2x6bcr0000gn/T/ipykernel_88335/2001734794.py:23: PerformanceWarning: indexing past lexsort depth may impact performance.
  dataset_result['total_score_{}'.format(dataset)] = dataset_result[(dataset, )].sum(axis=1)
/var/folders/v7/3s0lms795672_f7_mh2x6bcr0000gn/T/ipykernel_88335/2001734794.py:23: PerformanceWarning: indexing past lexsort depth may impact performance.
  dataset_result['total_score_{}'.format(dataset)] = dataset_result[(dataset, )].sum(axis=1)
/var/folders/v7/3s0lms795672_f7_mh2x6bcr0000gn/T/ipykernel_88335/2001734794.py:23: PerformanceWarning: indexing past lexsort depth may impact performance.
  dataset_result['total_score_{}'.format(dataset)] = dataset_result[(dataset, )].sum(axis=1)
/var/fol

In [75]:
import pandas as pd

# 1. Load the CSV (path is mounted in this notebook environment)
df = pd.read_csv('./discontinuity.csv')

# 2. --- Clean & reshape ----------------------------------------------------
# Drop any unnamed index column if present
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Identify the columns that uniquely describe a *configuration*
config_cols = [
    "model_type",
    "model_size",
    "pre_training_strategy",
    "loss_function",
    "backbone",
]

# Identify the dataset‑specific score columns (they all start with this prefix)
score_cols = [c for c in df.columns if c.startswith("total_score_")]

# Melt to long format: one row per (config, dataset, ft_strategy)
long_df = df.melt(
    id_vars=config_cols + ["ft_strategy"],
    value_vars=score_cols,
    var_name="dataset",
    value_name="score",
)

# Tidy the dataset names (remove the "total_score_" prefix)
long_df["dataset"] = long_df["dataset"].str.replace("total_score_", "")

# Pivot so each (dataset × ft_strategy) becomes its own column
pivot_df = long_df.pivot_table(
    index=config_cols,            # rows = unique configurations
    columns=["dataset", "ft_strategy"],
    values="score",
)

# Keep only the configurations that have *all* scores present
pivot_df_complete = pivot_df.dropna()
pivot_df_complete.to_csv("./discontinuity_per_dataset.csv")

filtered = pivot_df_complete.xs(
    key="supervised (robust)",
    level="pre_training_strategy",
    drop_level=False        # keep the level in the index; omit if you want it removed
)
filtered

dataset                                                                                              caltech101  \
ft_strategy                                                                                      FFT (5 epochs)   
model_type          model_size pre_training_strategy loss_function backbone                                       
fully attention     large      supervised (robust)   CLASSIC_AT    robust_vit_base_patch16_224            725.0   
                                                     TRADES_v2     robust_vit_base_patch16_224            624.0   
                    medium     supervised (robust)   CLASSIC_AT    robust_deit_small_patch16_224          699.0   
                                                     TRADES_v2     robust_deit_small_patch16_224          780.0   
fully convolutional large      supervised (robust)   CLASSIC_AT    robust_convnext_base                   814.0   
                                                     TRADES_v2     robust_convnext_base                   536.0   
                    medium     supervised (robust)   CLASSIC_AT    robust_convnext_tiny                   398.0   
                                                                   robust_resnet50                        491.0   
                                                     TRADES_v2     robust_convnext_tiny                   213.0   
                                                                   robust_resnet50                        760.0   

dataset                                                                                                           \
ft_strategy                                                                                      FFT (50 epochs)   
model_type          model_size pre_training_strategy loss_function backbone                                        
fully attention     large      supervised (robust)   CLASSIC_AT    robust_vit_base_patch16_224              79.0   
                                                     TRADES_v2     robust_vit_base_patch16_224             106.0   
                    medium     supervised (robust)   CLASSIC_AT    robust_deit_small_patch16_224           100.0   
                                                     TRADES_v2     robust_deit_small_patch16_224           112.0   
fully convolutional large      supervised (robust)   CLASSIC_AT    robust_convnext_base                     59.0   
                                                     TRADES_v2     robust_convnext_base                     45.0   
                    medium     supervised (robust)   CLASSIC_AT    robust_convnext_tiny                    515.0   
                                                                   robust_resnet50                         224.0   
                                                     TRADES_v2     robust_convnext_tiny                     93.0   
                                                                   robust_resnet50                         234.0   

dataset                                                                                                          \
ft_strategy                                                                                      LP (50 epochs)   
model_type          model_size pre_training_strategy loss_function backbone                                       
fully attention     large      supervised (robust)   CLASSIC_AT    robust_vit_base_patch16_224            825.0   
                                                     TRADES_v2     robust_vit_base_patch16_224            865.0   
                    medium     supervised (robust)   CLASSIC_AT    robust_deit_small_patch16_224          334.0   
                                                     TRADES_v2     robust_deit_small_patch16_224          391.0   
fully convolutional large      supervised (robust)   CLASSIC_AT    robust_convnext_base                   669.0   
                                                     TRADES_v2     robust_c

In [76]:
import pandas as pd

# ── start from the wide matrix you built earlier ────────────────────────────
#   rows:   model_type · model_size · pre_training_strategy · loss_function · backbone
#   cols:   MultiIndex (dataset, ft_strategy)
#   vals:   borda score
#   DataFrame name: pivot_df_complete
# ────────────────────────────────────────────────────────────────────────────

# 1. Make sure the column MultiIndex levels are named
if filtered.columns.names == [None, None]:
    filtered.columns.names = ["dataset", "ft_strategy"]

# 2. Stack (unpivot) the two column levels → long form
long_df = (
    filtered
        .stack(level=["dataset", "ft_strategy"])   # brings those two levels into the index
        .rename("borda")                           # give the stacked values a name
        .reset_index()                             # turn *all* index levels into columns
)

# long_df now looks like:
# ┌───────────┬───────────┬────────────────────────────┬──────────────┬──────────┬───────────────┬────────────┬────────┐
# │model_type │model_size │pre_training_strategy       │loss_function │backbone  │dataset        │ft_strategy │ borda │
# ├───────────┼───────────┼────────────────────────────┼──────────────┼──────────┼───────────────┼────────────┼────────┤
# │  …        │   …       │ supervised (robust)        │ CLASSIC_AT   │ r50_rob  │ stanford_cars │ FFT-5      │ 4032  │
# │  …        │   …       │ supervised (robust)        │ CLASSIC_AT   │ r50_rob  │ stanford_cars │ FFT-50     │ 4215  │
# │  …        │   …       │ …                          │ …            │ …        │ oxford_pet    │ FFT-5      │ 3770  │
# │  …        │   …       │ …                          │ …            │ …        │ …             │ …          │ …     │
# └───────────┴───────────┴────────────────────────────┴──────────────┴──────────┴───────────────┴────────────┴────────┘

# Optional: enforce a specific column order
ordered_cols = [
    "model_type", "model_size", "pre_training_strategy",
    "loss_function", "backbone", "dataset", "ft_strategy", "borda",
]
long_df = long_df[ordered_cols]

# ── now `long_df` has one numeric column (`borda`) and everything else is metadata ──
# print(long_df.head())
long_df
import numpy as np

conditions = [
    (long_df["backbone"] == "robust_resnet50") & (long_df["loss_function"] == "CLASSIC_AT"),
    (long_df["loss_function"] == "CLASSIC_AT"),
    (long_df["loss_function"] == "TRADES_v2"),
]

choices = [False, True, False]

long_df["continuity"] = np.select(conditions, choices, default=np.nan)
long_df.to_csv("discontinuity.csv")
long_df.head(50)

,model_type,model_size,pre_training_strategy,loss_function,backbone,dataset,ft_strategy,borda,continuity
0,fully attention,large,supervised (robust),CLASSIC_AT,robust_vit_base_patch16_224,caltech101,FFT (5 epochs),725.0,1.0
1,fully attention,large,supervised (robust),CLASSIC_AT,robust_vit_base_patch16_224,caltech101,FFT (50 epochs),79.0,1.0
2,fully attention,large,supervised (robust),CLASSIC_AT,robust_vit_base_patch16_224,caltech101,LP (50 epochs),825.0,1.0
3,fully attention,large,supervised (robust),CLASSIC_AT,robust_vit_base_patch16_224,fgvc-aircraft-2013b,FFT (5 epochs),773.0,1.0
4,fully attention,large,supervised (robust),CLASSIC_AT,robust_vit_base_patch16_224,fgvc-aircraft-2013b,FFT (50 epochs),423.0,1.0
5,fully attention,large,supervised (robust),CLASSIC_AT,robust_vit_base_patch16_224,fgvc-aircraft-2013b,LP (50 epochs),658.0,1.0
6,fully attention,large,supervised (robust),CLASSIC_AT,robust_vit_base_patch16_224,flowers-102,FFT (5 epochs),605.0,1.0
7,fully attention,large,supervised (robust),CLASSIC_AT,robust_vit_base_patch16_224,flowers-102,FFT (50 epochs),457.0,1.0
8,fully attention,large,supervised (robust),CLASSIC_AT,robust_vit_base_patch16_224,flowers-102,LP (50 epochs),605.0,1.0
9,fully attention,large,supervised (robust),CLASSIC_AT,robust_vit_base_patch16_224,oxford-iiit-pet,FFT (5 epochs),682.0,1.0


In [114]:
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu
import numpy as np

# Filter the subset

for ft_strategfy in [ "LP (50 epochs)", "FFT (50 epochs)", "FFT (5 epochs)" ]:
    sub = long_df[long_df["ft_strategy"] == ft_strategfy]
    print("########################", ft_strategfy, "\n")

    # Prepare to collect stats
    results = []

    # Group by dataset
    for dataset, group in sub.groupby("dataset"):
        group0 = group[group["continuity"] == False]["borda"]
        group1 = group[group["continuity"] == True]["borda"]

        if len(group0) > 1 and len(group1) > 1:
            # t-test (assumes normality)
            t_stat, t_p = ttest_ind(group0, group1, equal_var=False)
            # Mann-Whitney U test (non-parametric)
            u_stat, u_p = mannwhitneyu(group0, group1, alternative='two-sided')
            # Cohen's d
            pooled_std = np.sqrt((group0.std()**2 + group1.std()**2) / 2)
            cohens_d = (group1.mean() - group0.mean()) / pooled_std

            results.append({
                "dataset": dataset,
                "mean_discont": group1.mean(),
                "mean_cont": group0.mean(),
                "t_pval": t_p,
                "u_pval": u_p,
                "cohens_d": cohens_d
            })

    df_results = pd.DataFrame(results)

    group0_all = sub[sub["continuity"] == False]["borda"]
    group1_all = sub[sub["continuity"] == True]["borda"]

    # t-test
    t_stat, t_p = ttest_ind(group0_all, group1_all, equal_var=False)
    # Mann-Whitney U
    u_stat, u_p = mannwhitneyu(group0_all, group1_all, alternative='two-sided')
    # Cohen's d
    pooled_std = np.sqrt((group0_all.std()**2 + group1_all.std()**2) / 2)
    cohens_d = (group1_all.mean() - group0_all.mean()) / pooled_std

    print("Global t-test p-value:", t_p)
    print("Global Mann-Whitney p-value:", u_p)
    print("Global Cohen's d:", cohens_d)
    print("  ")

    top_k = [1, 3]

    for n in top_k:
        print(f"\n--- Top-{n} Analysis ---")
        top_k_stats = []

        for dataset, group in sub.groupby("dataset"):
            top = group.sort_values("borda", ascending=False).head(n)
            n_disc = (top["continuity"] == True).sum()
            n_cont = (top["continuity"] == False).sum()

            top_k_stats.append({
                "dataset": dataset,
                f"n_discontinuity_in_top{n}": n_disc,
                f"prop_discontinuity_in_top{n}": n_disc / n,
                f"n_continuity_in_top{n}": n_cont,
                f"prop_continuity_in_top{n}": n_cont / n,
            })

        df_topk = pd.DataFrame(top_k_stats)
        dataframe = df_topk  # optional: for display in notebook

        # Global top-k across datasets
        top_all = sub.sort_values(["dataset", "borda"], ascending=[True, False]).groupby("dataset").head(n)
        n_total = len(top_all)
        from statsmodels.stats.proportion import binom_test


        # Discontinuity stats
        n_disc = (top_all["continuity"] == True).sum()
        p_null_disc = sub["continuity"].mean()
        pval_disc = binom_test(count=n_disc, nobs=n_total, prop=p_null_disc, alternative="larger")

        # Continuity stats
        n_cont = (top_all["continuity"] == False).sum()
        p_null_cont = (sub["continuity"] == False).mean()
        pval_cont = binom_test(count=n_cont, nobs=n_total, prop=p_null_cont, alternative="larger")

        # Report
        print(f"\nDiscontinuity overrepresentation (continuity=True):")
        print(f"  Observed: {n_disc}/{n_total} = {n_disc / n_total:.2f}")
        print(f"  Expected: {p_null_disc:.2f}")
        print(f"  Binomial p-value: {pval_disc:.4f}")

        print(f"\nContinuity overrepresentation (continuity=False):")
        print(f"  Observed: {n_cont}/{n_total} = {n_cont / n_total:.2f}")
        print(f"  Expected: {p_null_cont:.2f}")
        print(f"  Binomial p-value: {pval_cont:.4f}")

    borda_sums = sub.groupby(["backbone", "loss_function", "ft_strategy", "continuity"])["borda"].sum().reset_index()
    borda_sums = borda_sums.sort_values("borda", ascending=False)

    from scipy.stats import ttest_ind, mannwhitneyu
    import numpy as np

    group0 = borda_sums[borda_sums["continuity"] == False]["borda"]
    group1 = borda_sums[borda_sums["continuity"] == True]["borda"]

    # t-test
    t_stat, t_p = ttest_ind(group0, group1, equal_var=False)

    # Mann-Whitney U test (non-parametric)
    u_stat, u_p = mannwhitneyu(group0, group1, alternative="two-sided")

    # Cohen's d
    pooled_std = np.sqrt((group0.std()**2 + group1.std()**2) / 2)
    cohens_d = (group1.mean() - group0.mean()) / pooled_std
    print(" ")
    print(f"T-test p-value: {t_p:.4f}")
    print(f"Mann-Whitney p-value: {u_p:.4f}")
    print(f"Cohen's d: {cohens_d:.2f}")

######################## LP (50 epochs) 

Global t-test p-value: 0.9368162894017608
Global Mann-Whitney p-value: 0.9699062667897117
Global Cohen's d: 0.020964338173966042
  

--- Top-1 Analysis ---

Discontinuity overrepresentation (continuity=True):
  Observed: 1/6 = 0.17
  Expected: 0.40
  Binomial p-value: 0.9533

Continuity overrepresentation (continuity=False):
  Observed: 5/6 = 0.83
  Expected: 0.60
  Binomial p-value: 0.2333

--- Top-3 Analysis ---

Discontinuity overrepresentation (continuity=True):
  Observed: 8/18 = 0.44
  Expected: 0.40
  Binomial p-value: 0.4366

Continuity overrepresentation (continuity=False):
  Observed: 10/18 = 0.56
  Expected: 0.60
  Binomial p-value: 0.7368
 
T-test p-value: 0.9640
Mann-Whitney p-value: 0.9143
Cohen's d: 0.03
######################## FFT (50 epochs) 

Global t-test p-value: 0.3466942048756243
Global Mann-Whitney p-value: 0.4195068723315438
Global Cohen's d: 0.2539197996611508
  

--- Top-1 Analysis ---

Discontinuity overrepresentatio